---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def degree_distribution(G):
    degrees = G.degree()
    degree_values = sorted(set(degrees.values()))
    histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes( G)) for i in degree_values]
    return histogram
#degree_distribution(P1_Graphs[1])

In [4]:
def graph_identification():
    iden = []
    for G in P1_Graphs:
        cluster = nx.average_clustering(G)
        shortest = nx.average_shortest_path_length(G)
        hist = degree_distribution(G)
        if len(hist)>10:
            iden.append('PA')
        elif cluster < 0.1:
            iden.append('SW_H')
        else:
            iden.append('SW_L')
    return iden
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [6]:

from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
#G.nodes(data=True)[0][1]

In [7]:

def salary_predictions():
    
    # Your Code Here
    def is_management(node):
        managementSalary = node[1]['ManagementSalary']
        if managementSalary == 0:
            return 0
        elif managementSalary == 1:
            return 1
        else:
            return None
        
    data = pd.DataFrame(index=G.nodes())
    data['cluster'] = pd.Series(nx.clustering(G))
    data['degree'] = pd.Series(G.degree())
    data['centrality'] = pd.Series(nx.degree_centrality(G))
    data['closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    data['betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    data['pr'] = pd.Series(nx.pagerank(G))
    data['is_management'] = pd.Series([is_management(node) for node in G.nodes(data=True)])
    data_train = data[~pd.isnull(data['is_management'])]
    data_test = data[pd.isnull(data['is_management'])]
    features = ['cluster', 'degree', 'centrality', 'closeness', 'betweeness', 'pr']
    X_train = data_train[features]
    Y_train = data_train['is_management']
    X_test = data_test[features]
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf = MLPClassifier(hidden_layer_sizes = [10, 5], alpha = 5,
                       random_state = 0, solver='lbfgs', verbose=0)
    clf.fit(X_train_scaled, Y_train)
    test_proba = clf.predict_proba(X_test_scaled)[:, 1]
    probability = pd.Series(test_proba,X_test.index)
    
    return probability


#salary_predictions()

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [8]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

Future Connection
(6, 840)                  0.0
(4, 197)                  0.0
(620, 979)                0.0
(519, 872)                0.0
(382, 423)                0.0
(97, 226)                 1.0
(349, 905)                0.0
(429, 860)                0.0
(309, 989)                0.0
(468, 880)                0.0

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [9]:

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
#G.node[1]

In [10]:
def new_connections_predictions():
    
    # Your Code Here
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
    pref_attachment = list(nx.preferential_attachment(G))
    data = pd.DataFrame(index=[(x[0], x[1]) for x in pref_attachment])
    data['pref_attachment'] = [x[2] for x in pref_attachment]
    soundarajan_hopcroft = list(nx.cn_soundarajan_hopcroft(G))
    data_soundarajan_hopcroft = pd.DataFrame(index=[(x[0], x[1]) for x in soundarajan_hopcroft])
    data_soundarajan_hopcroft['soundarajan_hopcroft'] = [x[2] for x in soundarajan_hopcroft]
    data = data.join(data_soundarajan_hopcroft,how='outer')
    data['soundarajan_hopcroft'] = data['soundarajan_hopcroft'].fillna(value=0)
    data['resource_allocation_index'] = [x[2] for x in list(nx.resource_allocation_index(G))]
    data['jaccard_coefficient'] = [x[2] for x in list(nx.jaccard_coefficient(G))]
    data = future_connections.join(data,how='outer')
    data_train = data[~pd.isnull(data['Future Connection'])]
    data_test = data[pd.isnull(data['Future Connection'])]
    features = ['soundarajan_hopcroft', 'pref_attachment', 'resource_allocation_index', 'jaccard_coefficient']
    X_train = data_train[features]
    Y_train = data_train['Future Connection']
    X_test = data_test[features]
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf = MLPClassifier(hidden_layer_sizes = [10, 5], alpha = 5,
                       random_state = 0, solver='lbfgs', verbose=0)
    clf.fit(X_train_scaled, Y_train)
    test_proba = clf.predict_proba(X_test_scaled)[:, 1]
    predictions = pd.Series(test_proba,X_test.index)
    target = future_connections[pd.isnull(future_connections['Future Connection'])]
    target['probability'] = [predictions[x] for x in target.index]
    return target['probability']

    
new_connections_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(107, 348)    0.030798
(542, 751)    0.012661
(20, 426)     0.590189
(50, 989)     0.012774
(942, 986)    0.012896
(324, 857)    0.012758
(13, 710)     0.157992
(19, 271)     0.107565
(319, 878)    0.012813
(659, 707)    0.012673
(49, 843)     0.012848
(208, 893)    0.012700
(377, 469)    0.008606
(405, 999)    0.021212
(129, 740)    0.018759
(292, 618)    0.023529
(239, 689)    0.012821
(359, 373)    0.010155
(53, 523)     0.076263
(276, 984)    0.012843
(202, 997)    0.012866
(604, 619)    0.047400
(270, 911)    0.012819
(261, 481)    0.063398
(200, 450)    0.997548
(213, 634)    0.012626
(644, 735)    0.041949
(346, 553)    0.012347
(521, 738)    0.011817
(422, 953)    0.019543
                ...   
(672, 848)    0.012819
(28, 127)     0.977766
(202, 661)    0.012300
(54, 195)     0.997570
(295, 864)    0.012741
(814, 936)    0.012643
(839, 874)    0.012896
(139, 843)    0.012750
(461, 544)    0.011453
(68, 487)     0.011390
(622, 932)    0.012713
(504, 936)    0.017593
(479, 528) 